## Loading Data

In [3]:
import pandas as pd
import os

In [34]:
if not os.path.exists("data"):
    os.makedirs("data")
    
dfs = {}
for filename in os.listdir("data"):
    if not filename.endswith(".csv"):
        continue
    if "combined" in filename:
        continue
    
    path = os.path.join("data", filename)
    try:
        df = pd.read_csv(path, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding="cp1252")
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
    df['Item'] = df['Item'].astype(str)
    df['Category'] = df['Category'].astype(str)
    
    key = filename.removesuffix(".csv")
    dfs[key] = df
    print(f"Loaded {filename} with {len(df)} rows. Null Entries: {df.isnull().sum().sum()}")

Loaded 2018.csv with 112 rows. Null Entries: 0
Loaded 2019.csv with 154 rows. Null Entries: 0
Loaded 2020.csv with 109 rows. Null Entries: 0
Loaded 2021.csv with 168 rows. Null Entries: 0
Loaded 2022.csv with 135 rows. Null Entries: 0
Loaded 2023.csv with 296 rows. Null Entries: 0
Loaded 2024.csv with 216 rows. Null Entries: 0


In [31]:
combined = []
for year, df in dfs.items():
    df = df.copy()
    combined.append(df)

df_combined = pd.concat(combined, ignore_index=True)
df_combined.info()
df_combined.head()
df_combined.to_csv("data/combined.csv", index=False, encoding="utf-8")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Item      1190 non-null   object        
 1   Category  1190 non-null   object        
 2   Cost      1190 non-null   float64       
 3   Date      1190 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 37.3+ KB
